# Cycle Simulations

What are they?

This example is run in docker



In [1]:
import datetime
import os
import pandas as pd
import pathlib
import pickle
from pprint import pprint
import shlex
import subprocess
import sys
import wrfhydropy

/home/docker/miniconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/docker/miniconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
model_repo = pathlib.Path('/wrf_hydro_nwm_public')
experiment_dir = pathlib.Path('/home/docker/wrfhydropy_cycle_example')
if not experiment_dir.exists():
    os.mkdir(str(experiment_dir))
os.chdir(str(experiment_dir))

In [3]:
domain_dir = experiment_dir / 'domain'
if not domain_dir.exists(): 
    sys.path.append(str(model_repo / 'tests/local/utils'))
    from gdrive_download import download_file_from_google_drive
    file_id = '1xFYB--zm9f8bFHESzgP5X5i7sZryQzJe'
    file_target = 'gdrive_testcase.tar.gz'
    download_file_from_google_drive(
        file_id, 
        str(experiment_dir.joinpath(file_target)))
    untar_cmd = 'tar -xf ' + file_target + '; mv example_case domain'
    subprocess.run(
        untar_cmd, 
        shell=True,
        cwd=str(experiment_dir))

In [4]:
hrldas_model_side_file = model_repo / 'trunk/NDHMS/hrldas_namelists.json'
hrldas_domain_side_file = domain_dir / 'hrldas_namelist_patches.json'
hydro_model_side_file = model_repo / 'trunk/NDHMS/hydro_namelists.json'
hydro_domain_side_file = domain_dir / 'hydro_namelist_patches.json'
compile_options_file = model_repo / 'trunk/NDHMS/compile_options.json'
config = 'nwm_ana'

In [5]:
domain = wrfhydropy.Domain(
    domain_top_dir=domain_dir,
    domain_config=config)

In [35]:
model = wrfhydropy.Model(
    model_repo / 'trunk/NDHMS',
    compiler='gfort',
    model_config=config)

In [34]:
restart_file = domain.__dict__['hydro_namelist_patches']['hydro_nlist']['restart_file']
domain.__dict__['hydro_namelist_patches']['hydro_nlist']['restart_file'] = domain_dir / restart_file

In [36]:
compile_dir = experiment_dir / 'compile'
if not compile_dir.exists():
    model.compile(compile_dir)
else: 
    model = pickle.load(compile_dir.joinpath('WrfHydroModel.pkl').open('rb'))
    # Caveat here, the config used for compiling is baked into the restored model object. A change/fix to this is easy to implement... 

In [39]:
exe_cmd = 'mpirun -np 1 ./wrf_hydro.exe'
model_start_time = datetime.datetime(2011, 8, 26)
model_end_time = model_start_time + datetime.timedelta(hours=24)
job_24hr = wrfhydropy.Job(
    job_id='24hr',
    exe_cmd=exe_cmd,
    model_start_time=model_start_time,
    model_end_time=model_end_time,
    restart=True,
    restart_freq_hr=6,
    output_freq_hr=1)

simulation = wrfhydropy.Simulation()
simulation.add(model)
simulation.add(domain)
simulation.add(job_24hr)

sim_dir = experiment_dir / 'simulation'
os.mkdir(sim_dir)
os.chdir(sim_dir)
simulation.compose()

FileExistsError: [Errno 17] File exists: '/home/docker/wrfhydropy_cycle_example/simulation'

In [40]:
simulation.run()


Running job 24hr: 
    Wall start time: 2020-04-08 02:54:48
    Model start time: 2011-08-26 00:00
    Model end time: 2011-08-27 00:00


# Cycles

In [41]:
help(wrfhydropy.CycleSimulation)

Help on class CycleSimulation in module wrfhydropy.core.cycle:

class CycleSimulation(builtins.object)
 |  CycleSimulation(init_times: list, restart_dirs: list, forcing_dirs: list = [], ncores: int = 1)
 |  
 |  Class for a WRF-Hydro CycleSimulation object. The Cycle Simulation object is used to
 |  orchestrate a set of 'N' WRF-Hydro simulations, referred to as 'casts', which only differ
 |  in their 1) restart times and 2) their forcings.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, init_times: list, restart_dirs: list, forcing_dirs: list = [], ncores: int = 1)
 |      Instantiates an EnsembleSimulation object.
 |  
 |  __len__(self)
 |  
 |  add(self, obj: Union[wrfhydropy.core.simulation.Simulation, wrfhydropy.core.schedulers.Scheduler, wrfhydropy.core.job.Job])
 |      Add an approparite object to an EnsembleSimulation, such as a Simulation, Job, or
 |      Scheduler.
 |      Args:
 |          obj: the object to add.
 |  
 |  compose(self, symlink_domain: bool = True, for

In [42]:
init_times = [datetime.datetime(2011, 8, 26, 0) + 
              datetime.timedelta(hours=hh) for hh in range(0, 24, 6)]
init_times

[datetime.datetime(2011, 8, 26, 0, 0),
 datetime.datetime(2011, 8, 26, 6, 0),
 datetime.datetime(2011, 8, 26, 12, 0),
 datetime.datetime(2011, 8, 26, 18, 0)]

In [48]:
restart_dirs = [sim_dir] * len(init_times)

In [49]:
restart_dirs

[PosixPath('/home/docker/wrfhydropy_cycle_example/simulation'),
 PosixPath('/home/docker/wrfhydropy_cycle_example/simulation'),
 PosixPath('/home/docker/wrfhydropy_cycle_example/simulation'),
 PosixPath('/home/docker/wrfhydropy_cycle_example/simulation')]

In [45]:
job_cycle = wrfhydropy.Job(
    job_id='cycle',
    exe_cmd=exe_cmd,
    model_start_time=init_times[0],
    model_end_time=init_times[0] + datetime.timedelta(hours=3),
    restart=True,
    restart_freq_hr=-99999,
    output_freq_hr=1)

cycle = wrfhydropy.CycleSimulation(
    init_times=init_times, restart_dirs=restart_dirs)

cycle.add(simulation)
cycle.add(job_cycle)

In [50]:
cycle._restart_dirs

[PosixPath('/home/docker/wrfhydropy_cycle_example/simulation'),
 PosixPath('.'),
 PosixPath('.'),
 PosixPath('.')]

In [46]:
cycle_dir = experiment_dir / 'cycle'
os.mkdir(cycle_dir)
os.chdir(cycle_dir)
cycle.compose()

Composing simulation into directory:'/home/docker/wrfhydropy_cycle_example/cycle/cast_2011082600'
Getting domain files...
/home/docker/wrfhydropy_cycle_example/domain/NWM/DOMAIN/nudgingParams.nc
/home/docker/wrfhydropy_cycle_example/domain/NWM/RESTART/nudgingLastObs.2011-08-26_00:00:00.nc
Making job directories...
Validating job input files
cycle
Model already compiled, copying files...
Simulation successfully composed
Composing simulation into directory:'/home/docker/wrfhydropy_cycle_example/cycle/cast_2011082606'
Getting domain files...
/home/docker/wrfhydropy_cycle_example/domain/NWM/DOMAIN/nudgingParams.nc
/home/docker/wrfhydropy_cycle_example/domain/NWM/RESTART/nudgingLastObs.2011-08-26_00:00:00.nc
Making job directories...
Validating job input files
cycle
Model already compiled, copying files...
Simulation successfully composed
Composing simulation into directory:'/home/docker/wrfhydropy_cycle_example/cycle/cast_2011082612'
Getting domain files...
/home/docker/wrfhydropy_cycle_ex

In [47]:
cycle.run()


Running job cycle: 
    Wall start time: 2020-04-08 02:55:09
    Model start time: 2011-08-26 00:00
    Model end time: 2011-08-26 03:00

Running job cycle: 
    Wall start time: 2020-04-08 02:55:10
    Model start time: 2011-08-26 06:00
    Model end time: 2011-08-26 09:00


RuntimeError: Model did not finish successfully

In [18]:
# Collection

In [19]:
# Plot?